# Pràctica 4: preprocessament i extracció de característiques
En aquesta pràctica treballarem en el preprocessament de text i en l'extracció de característiques *sparse* i denses dels documents.

**Alumne 1**: <label style="color:green"> Alejandro Madrid Galarza </label>

**Alumne 2**: <label style="color:green"> Antonio José López Martínez </label>

In [4]:
import spacy
import pandas as pd
import numpy as np

nlp = spacy.load("es_core_news_sm")
nlp.add_pipe("merge_entities") # Ajuntem els tokens de cada entitat.

C:\Users\ajlmt\AppData\Local\Temp\ipykernel_14416\608376737.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

## Part 1: preprocessament de text
En aquesta part construirem una funció generadora per carregar els documents de text des d'una carpeta de disc, i realitzarem un processament inicial a cada document.  
Farem servir un conjunt de notícies esportives del Marca, obtingut de https://www.kaggle.com/datasets/mdamsterdam/marca-spanish-sports-news

In [5]:
import os

def carrega_textos(path):
    """Funció generadora que carrega els fitxers de tipus TXT d'una carpeta.
    Retorna (yield) el text següent en cada execució."""

    for file in [f for f in os.listdir(path) if f.endswith('.txt')]:
        with open(os.path.join(path, file), encoding='utf-8') as f:
            
            yield f.read()

Prova aquesta funció carregant el primer fitxer de la carpeta de materials de la pràctica (`"P4_materials/noticies"`).

In [6]:
ruta = "P4_materials/noticies"
gen = carrega_textos(ruta)
text = next(gen)
print(text[:900])

Campazzo y Tavares llevan al Real Madrid a su séptima final de Copa seguida El Unicaja ya está en "su" final tras aplastar al MoraBanc Ibai Llanos la "lía" allá por donde va. Esta vez ha sido con un lanzamiento desde medio campo. El youtuber, que andaba conversando con su amigo Facundo Campazzo a su llegada al Martín Carpena, ha vuelto a armarla.  QUIÉN ES CURRY NO ME SUENA DE NADA pic.twitter.com/8gS6V8JXqp Solo tienen que ver la reacción tras su canasta anotada desde el centro de la cancha. "¿Quién es Curry?¡No le conozco!", ha gritado. Un ídolo de masas.  Suscríbete a la Newsletter de Basket de MARCA y recibe en tu correo electrónico, de lunes a domingo y a primera hora de la mañana, las noticias exclusivas, entrevistas, reportajes, gráficos y vídeos que marcarán el día en la NBA, Liga Endesa, Euroliga y el resto del mundo de la canasta.


### Normalització de text.
Crea una funció per normalitzar el text de cada notícia. Els passos a realitzar per la funció seran:  
- Eliminar *stop-words* i signes de puntuació.
- Extreure el lema de cada paraula.
- Anonimitzar el text (substituir les entitats de tipus `PER` pel text "PERSONA").  

Utilitza la llibreria `spacy` amb el model `ES` per fer la normalització. La funció accepta com a entrada un *string* i torna a la sortida també un *string*.

In [7]:
from spacy.lang.es.stop_words import STOP_WORDS
import string

def normalitza(text):
    """Funció que normalitza un string de text.
    Entrada: string a normalitzar.
    Retorna: string del text normalitzat.
"""
    # Processament del text amb SpaCy
    doc = nlp(text)
    
    # Eliminació de stop-words i signes de puntuació, lematització i anonimització
    tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        if token.ent_type_ == 'PER':
            tokens.append('PERSONA')
        else:
            tokens.append(token.lemma_)
    
    # Retornem el text normalitzat
    return ' '.join(tokens)


Proveu aquesta funció sobre la primera notícia de la carpeta.

In [8]:
ruta = "P4_materials/noticies"
gen = carrega_textos(ruta)
text = next(gen)

normalitzat = normalitza(text)
print("Text normalitzat:")
print(normalitzat)

Text normalitzat:
PERSONA PERSONA llevar Real Madrid séptimo Copa seguido el Unicaja aplastar MoraBanc Ibai Llanos lía allá lanzamiento campo el youtuber andar conversar amigo PERSONA llegada PERSONA volver armar él   CURRY SUENA pic.twitter.com/8gs6v8jxqp reacción canasta anotado centro cancha ¿quién ser Curry?¡No él conocer! gritar uno ídolo de masa   Suscríbete a el Newsletter de Basket MARCA recibir correo electrónico lunes domingo hora mañana noticia exclusivo entrevista reportaje gráfico vídeo marcar NBA Liga Endesa Euroliga resto mundo canasta


## Part 2: extracció de característiques globals.
Extreurem un conjunt de característiques globals de cada document:  
- Longitud del text.
- Nombre de paraules i frases.
- Nombre d'entitats a cada text.

### Extracció de característiques.
Crea una funció que a partir d'un text d'entrada (objecte `string`) genere un diccionari de característiques amb els valors següents:  
- `caracters`: longitud del text en caràcters.
- `paraules`: nombre de paraules del text excloent tokens de puntuació.
- `frases`: nombre de frases del text.
- `ENT_PER`: nº d'entitats de tipus `PER` al text.
- `ENT_LOC`: nº d'entitats de tipus `LOC` al text
- `ENT_ORG`: nº d'entitats de tipus `ORG` al text.
- `ENT_MISC`: nº d'entitats de tipus `MISC` al text.  

Ajuda: utilitza la classe `Counter` de la llibreria `collections` per comptar el nombre d'entitats de cada tipus.

In [9]:
from collections import Counter

def caracteristiques(text):
    """Calcula una sèrie de característiques d'un text i les retorna com a valors d'un objecte diccionari."""
    
    doc = nlp(text)
    entitats = Counter([ent.label_ for ent in doc.ents])
    paraules = [token.text.lower() for token in doc if not token.is_punct]
    num_paraules = len(paraules)
    num_frases = len(list(doc.sents))
    
    carac_dict = {
        'caracters': len(text),
        'paraules': num_paraules,
        'frases': num_frases,
        'ENT_PER': entitats['PER'],
        'ENT_LOC': entitats['LOC'],
        'ENT_ORG': entitats['ORG'],
        'ENT_MISC': entitats['MISC']
    }
    
    return carac_dict


Prova-ho sobre el primer text.

In [10]:
ruta = "P4_materials/noticies"
gen = carrega_textos(ruta)
text = next(gen)

carac = caracteristiques(text)
print("Text caracteritzat:")
print(carac)

Text caracteritzat:
{'caracters': 852, 'paraules': 132, 'frases': 8, 'ENT_PER': 4, 'ENT_LOC': 2, 'ENT_ORG': 4, 'ENT_MISC': 8}


Crea un objecte DataFrame de `pandas` amb aquestes característiques per a tots els textos dins la carpeta de materials.

In [11]:
import os

textos = list(carrega_textos("P4_materials/noticies"))
caracteristiques_texts = []

for text in textos:
    carac = caracteristiques(text)
    caracteristiques_texts.append(carac)

df_caracteristiques = pd.DataFrame(caracteristiques_texts)
print(df_caracteristiques.head())


   caracters  paraules  frases  ENT_PER  ENT_LOC  ENT_ORG  ENT_MISC
0        852       132       8        4        2        4         8
1        716       112       4        3        3        5         6
2        725       119       4        2        1        4         6
3       4378       669      45       23       15        2        32
4       3214       502      33       17       10        4        20


Analitza les estadístiques d'aquest DataFrame.

In [12]:
estadistiques = df_caracteristiques.describe()
print(estadistiques)

          caracters     paraules      frases     ENT_PER     ENT_LOC  \
count    561.000000   561.000000  561.000000  561.000000  561.000000   
mean    2429.572193   379.433155   19.146168   14.775401   10.718360   
std     1652.645616   257.481926   20.291713   15.695979   10.192741   
min      124.000000    22.000000    1.000000    0.000000    0.000000   
25%     1429.000000   222.000000    8.000000    6.000000    4.000000   
50%     2072.000000   327.000000   14.000000   11.000000    8.000000   
75%     3017.000000   468.000000   21.000000   18.000000   14.000000   
max    16002.000000  2297.000000  196.000000  180.000000   66.000000   

          ENT_ORG    ENT_MISC  
count  561.000000  561.000000  
mean     4.427807   14.641711  
std      4.537803   14.956901  
min      0.000000    0.000000  
25%      2.000000    7.000000  
50%      3.000000   11.000000  
75%      6.000000   16.000000  
max     41.000000  137.000000  


## Part 3: extracció de característiques *sparse*.
Obtindrem una matriu Bag-of-Words i TF-IDF sobre el corpus anterior (conjunt de notícies del Marca), després de fer una neteja prèvia a cada document.

In [13]:
nlp = spacy.load("es_core_news_sm") # Carreguem per eliminar el merge_entities.

### Neteja del text
Defineix una funció de neteja sobre un string que faça les accions següents:  
- Eliminar stop-words i signes de puntuació.  
- Eliminar aquelles paraules el `POS` de les quals no sigui ni nom, adjectiu o verb.
- Extreure el lema de cada paraula.
- Convertir en minúscula.

In [14]:
def neteja(text):
    """Realitza una neteja d'un text i torna el text netejat com a string"""
    doc = nlp(text)

    paraules_excloure = set()
    for token in doc:
        if not token.pos_ in ['NOUN', 'ADJ', 'VERB']:
            paraules_excloure.add(token.text.lower())
    
    tokens_net = []
    for token in doc:
        if token.is_stop or token.is_punct or token.text.lower() in paraules_excloure:
            continue
        tokens_net.append(token.lemma_.lower())
    
    return ' '.join(tokens_net)


Proveu la funció sobre el primer article de notícies.

In [15]:
ruta = "P4_materials/noticies"
gen = carrega_textos(ruta)
text = next(gen)

net = neteja(text)
print("Text netejat:")
print(net)

Text netejat:
llevar séptimo seguido aplastar lía lanzamiento campo youtuber andar conversar amigo llegada volver armar él pic.twitter.com/8gs6v8jxqp reacción canasta anotado centro cancha conocer gritar ídolo masa recibir correo electrónico lunes domingo hora mañana noticia exclusivo entrevista reportaje gráfico vídeo marcar resto mundo canasta


### Matriu Bag-of-Words.
Crea amb la llibreria `sklearn` la matriu BoW de tots els fitxers de notícies del directori de pràctiques a partir dels textos netejats. Utilitza la funció `map` per crear un iterador amb els fitxers netejats mitjançant la funció anterior.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

ruta = "P4_materials/noticies"
textos_net = map(lambda file: neteja(open(os.path.join(ruta, file), encoding='utf-8').read()), os.listdir(ruta))

vectoritzador = CountVectorizer()
matriu_bow = vectoritzador.fit_transform(textos_net)
print("Matriu Bag-of-Words")
print(matriu_bow)

ModuleNotFoundError: No module named 'sklearn'

Mostra les dimensions de la matriu generada.

In [ ]:
matriu_bow.shape

#### Anàlisi de les paraules més freqüents.
Calcula la freqüència d'ús de cada paraula (no la freqüència de documents) i guarda'l un DataFrame amb les columnes "paraules" i "freqüència", ordena-les de major a menor i mostra les 20 més freqüents.

In [ ]:
from ast import arg
vocabulario = vectoritzador.get_feature_names_out()

# Sumar las ocurrencias de cada palabra en toda la bolsa de palabras
frecuencia_palabras = matriu_bow.sum(axis=0)

# Obtener las frecuencias y palabras correspondientes
frecuencia_palabras = frecuencia_palabras.tolist()[0]
palabras = [vocabulario[i] for i in range(len(vocabulario))]

# Crear un DataFrame con las columnas "palabras" y "frecuencia"
df_frecuencia = pd.DataFrame({"palabras": palabras, "frecuencia": frecuencia_palabras})

# Ordenar el DataFrame por frecuencia en orden descendente
df_frecuencia = df_frecuencia.sort_values(by="frecuencia", ascending=False)

# Mostrar las 20 palabras más frecuentes
print(df_frecuencia.head(20))

## Part 4: extracció de característiques denses.
Utilitzarem els *word embeddings* de les paraules de cada document per generar un vector dens de document. Amb aquests vectors implementarem un classificador *zero-shot* sobre els documents.  
Utilitzarem ací un conjunt de dades que representa un carretó de compra, que intentarem associar a diferents categories representades per un text descriptiu.  

### Càrrega de *word embeddings*.
Farem servir els word embeddings en castellà de https://github.com/dccuchile/spanish-word-embeddings en format Word2Vec de Gensim.  
Carregueu els vectors fent servir la classe `KeyedVectors` de `Gensim` a l'objecte `wordvectors`.

Nota: Per a instal·lar el mòdul, utilitzeu `pip install -U gensim` una vegada activat el vostre entorn de `conda` i reinicieu el kernel.

In [ ]:
from gensim.models import KeyedVectors

# Ruta del fitxer de vectors de paraules
ruta_vectors = "path_to_vectors/SBW-vectors-300-min5.bin"  # Substitueix "path_to_vectors" per la ruta correcta del fitxer

# Carreguem els vectors de paraules
wordvectors = KeyedVectors.load_word2vec_format(ruta_vectors, binary=True)

# Exemple d'ús:
vector_rei = wordvectors["rei"]  # Vector de la paraula "rei"
print("Dimensionalitat del vector:", len(vector_rei))  # Imprimeix la dimensionalitat del vector


Com a exemple, busquem les paraules més similars semànticament a "automóvil" segons la seva similitud cosinus.

In [ ]:
wordvectors.most_similar(['automóvil'])

Definim una funció per calcular el *sentence embedding* d'un text com la mitjana dels *word embeddings* de les seves paraules.

In [ ]:
from numpy.linalg import norm # Per normalitzar dades.

def to_vector(text):
    """Calcula el vector dens del text com a mitjana dels word embeddings de les paraules"""
    tokens = text.lower().split()
    vec = np.zeros(300) # L'embedding té un tamany de '300'.
    for word in tokens:
        # Si la paraula està l'acumulem.
        if word in wordvectors:
            vec += wordvectors[word]
    return vec / norm(vec)

Prova aquesta funció per calcular la similitud cosinus entre les frases '*el partido de fútbol*', '*deportes de equipo*' i '*noticias internacionales*'.  
Defineix per a això una funció `similarity` sobre dos *strings* d'entrada usant l'operador producte matricial de NumPy (`@`).

In [ ]:
def similarity(text_1, text_2):
    """Calcula la similitud cosinus entre dos textos
    com el producte matricial dels seus vectors densos
    calculats amb la funció to_vector()"""
    
    vec_1 = to_vector(text_1)
    vec_2 = to_vector(text_2)
    cos_sim = vec_1 @ vec_2
    
    return cos_sim


In [ ]:
# Calcula la similitud entre els 3 textos.
texto_1 = "el partido de fútbol"
texto_2 = "deportes de equipo"
texto_3 = "noticias internacionales"

salida = similarity(texto_1, texto_2)
print(salida)
salida = similarity(texto_1, texto_3)
print(salida)
salida = similarity(texto_2, texto_3)
print(salida)


Modifica la funció `to_vector` per eliminar paraules que siguen dígits (mètode `str.isdigit()`) o la longitud de les quals siga menor a 3 caràcters.

In [ ]:
from numpy.linalg import norm

def to_vector(text):
    """Calcula el vector dens del text com a mitjana dels word embeddings de les paraules,
    eliminant paraules que siguin dígits o la longitud de les quals siga menor a 3 caràcters."""
    
    tokens = text.lower().split()
    vec = np.zeros(300) # L'embedding té una dimensió de '300'.
    
    for word in tokens:
        if word.isdigit() or len(word) < 3:
            continue
        
        if word in wordvectors:
            vec += wordvectors[word]
    
    if norm(vec) != 0:
        vec /= norm(vec)
    
    return vec


Comprova la similitud als 3 textos anteriors amb la nova funció de vectorització.

In [17]:
# Calcula la similitud entre els 3 textos.
texto_1 = "el partido de fútbol"
texto_2 = "deportes de equipo"
texto_3 = "noticias internacionales"

salida = to_vector(texto_1)
print(salida)
salida = to_vector(texto_3)
print(salida)
salida = to_vector(texto_2)
print(salida)


NameError: name 'to_vector' is not defined

## Ús de vectors densos en un problema de classificació *zero-shot*.
Carregarem un llistat de compres etiquetat al costat de la seua categoria, definida com una llista de termes. Farem servir la similitud cosinus entre cada compra i les categories per classificar-la automàticament.  
Les dades de les compres són al fitxer `dataset_compras.csv` i les seves classes al fitxer `classes.txt`. Carrega tots dos fitxers al DataFrame `compres` i la llista `classes` respectivament. L'ordre de les classes a l'arxiu es correspon a l'enter assignat a cada classe a `compres`.

Cada línia del fitxer *classes.txt* correspon a una etiqueta diferent. Hi ha un total de 7 línies, pel que cada línia correspondrà a un enter del CSV de les compres.

In [20]:
# Carreguem compres i classes.
compres = pd.read_csv("compres_subset.csv")

# Carreguem la llista de classes
with open("classes.txt", "r") as file:
    classes = file.readlines()

# Eliminem els caràcters de nova línia (\n) de cada element de la llista de classes
classes = [classe.strip() for classe in classes]

# Mostrem les primeres files del DataFrame compres i la llista classes
print("Primeres files del DataFrame compres:")
print(compres.head())
print("\nClasses:")
print(classes)


Primeres files del DataFrame compres:
                               compra  clase
0                      adhesivo mayon      4
1                     cerveza el lata      1
2  pasaje en avión santiago balmaceda      5
3  pasaje ida y vuelta cqob la serena      5
4                 reparación de rueda      5

Classes:
['alimentos comida bebida carne pollo jugo', 'alcohol cigarrillo tabaco', 'ropa de vestir calzado zapatos vestidos', 'muebles hogar aseo herramienta', 'salud medicamento hospital', 'transporte bus aviÃ³n automÃ³vil', 'comunicaciones telÃ©fono celular']


Agafa una compra a l'atzar i calcula la classe predita mitjançant la similitud cosinus entre el text de la compra i el text de cada classe.  
Nota: crea un array a NumPy amb les similituds cosinus de la compra a cada classe i calcula la posició del màxim amb `np.argmax()`.

### Càlcul de *doc embeddings*.
Genera els vectors de tots els ítems del DataFrame de compres i carrega'ls en un array 2D de NumPy usant `np.vstack()` anomenat `X`.  
Fes el mateix amb les classes en un array anomenat `Y`.  
Mostra la mida dels dos arrays.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report

# Calculem la similitud cosinus de cada compra amb totes les classes
similituds_cosinus = cosine_similarity(X, Y.T)

# Obtenim la classe predita per a cada compra com l'índex del màxim per files
prediccions_classe = np.argmax(similituds_cosinus, axis=1)

# Calculem les mètriques del classificador
print(classification_report(y_true, prediccions_classe, target_names=classes))


ModuleNotFoundError: No module named 'sklearn'

Calcula el producte matricial entre `X` i la transposada de `Y` per obtenir la similitud cosinus de cada compra a totes les classes.  
Obtin la predicció de classe assignada a cada compra com a índex del màxim per files (funció `np.argmax()`) i calcula les mètriques del classificador usant `classification_report` de la llibreria `sklearn`.

In [23]:
from sklearn.metrics import classification_report

# Calculem la similitud cosinus de cada compra amb totes les classes
similituds_cosinus = X @ Y.T

# Obtenim la classe predita per a cada compra com l'índex del màxim per files
prediccions_classe = np.argmax(similituds_cosinus, axis=1)

# Calculem les mètriques del classificador
print(classification_report(y_true, prediccions_classe, target_names=classes))


ModuleNotFoundError: No module named 'sklearn'

Repeteix la classificació usant el vectoritzador millorat `to_vector_plus` i compara els resultats.

In [ ]:
def to_vector_plus(text):
    """Calcula el vector dens del text com a mitjana dels word embeddings de les paraules,
    eliminant paraules que siguin dígits, la longitud de les quals siga menor a 3 caràcters
    i normalitzant el vector final."""
    
    tokens = text.lower().split()
    vec = np.zeros(300) # L'embedding té una dimensió de '300'.
    
    for word in tokens:
        # Si la paraula és un dígit o la seva longitud és menor a 3, la saltem.
        if word.isdigit() or len(word) < 3:
            continue
        
        # Si la paraula està en els word embeddings, l'acumulem.
        if word in wordvectors:
            vec += wordvectors[word]
    
    # Normalitzem el vector
    if norm(vec) != 0:
        vec /= norm(vec)
    
    return vec
